In [9]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, format_number, trim

base_path = r'G:\.shortcut-targets-by-id\1eDKYURr-Qm222Ul6PoZLc4b99cuwHTC3\Cường\Đại học\DS200\Lab 1'

spark = SparkSession.builder \
    .appName("DS200 Lab1 - Bai 3") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

try:
    movies_temp_df = spark.read.format("csv").option("header", "false") \
        .load(os.path.join(base_path, "movies.cleaned.txt")).toDF("MovieID_str", "Title_str", "Genres_str")
    movies_df = movies_temp_df.select(
        col("MovieID_str").cast("int").alias("MovieID"),
        col("Title_str").alias("Title"),
        col("Genres_str").alias("Genres")
    )

    ratings_temp_df = spark.read.format("csv").option("header", "false") \
        .load(os.path.join(base_path, "ratings.cleaned.txt")).toDF("UserID_str", "MovieID_str", "Rating_str", "Timestamp_str")
    ratings_df = ratings_temp_df.select(
        col("UserID_str").cast("int").alias("UserID"),
        col("MovieID_str").cast("int").alias("MovieID"),
        col("Rating_str").cast("float").alias("Rating"),
        col("Timestamp_str").cast("long").alias("Timestamp")
    )

    users_temp_df = spark.read.format("csv").option("header", "false") \
        .load(os.path.join(base_path, "users.cleaned.txt")).toDF("UserID_str", "Gender_str", "Age_str", "Occupation_str", "ZipCode_str")
    users_df = users_temp_df.select(
        col("UserID_str").cast("int").alias("UserID"),
        trim(col("Gender_str")).alias("Gender"),
        col("Age_str").cast("int").alias("Age"),
        col("Occupation_str").cast("int").alias("Occupation"),
        col("ZipCode_str").alias("ZipCode")
    )

except Exception as e:
    print(f"\nĐÃ CÓ LỖI XẢY RA KHI TẢI DỮ LIỆU: {e}")
    spark.stop()
    exit()

#  Join dữ liệu Ratings và Users

In [11]:
ratings_with_gender_df = ratings_df.join(users_df, "UserID")
ratings_with_gender_df.show(5)

+------+-------+------+---------+------+---+----------+-------+
|UserID|MovieID|Rating|Timestamp|Gender|Age|Occupation|ZipCode|
+------+-------+------+---------+------+---+----------+-------+
|   537|   1043|   4.0|964982703|     F| 25|        10|  48067|
|   642|   3791|   3.5|964982710|     M| 30|        15|  12345|
|   789|   4862|   5.0|964982720|     F| 22|         7|  67890|
|   556|   5930|   4.5|964982730|     M| 35|        12|  54321|
|   612|   5274|   3.0|964982740|     F| 28|        10|  98765|
+------+-------+------+---------+------+---+----------+-------+
only showing top 5 rows


# Group và Pivot theo Giới tính

In [12]:
gender_stats_df = ratings_with_gender_df.groupBy("MovieID") \
    .pivot("Gender", ["M", "F"]) \
    .agg(avg("Rating")) \
    .withColumnRenamed("M", "Male_Avg") \
    .withColumnRenamed("F", "Female_Avg")

gender_stats_df.show(5)

+-------+--------+----------+
|MovieID|Male_Avg|Female_Avg|
+-------+--------+----------+
|   3876|     4.0|       4.0|
|   4357|     4.5|       3.5|
|   2247|     3.5|       3.5|
|   4782|     4.5|       3.0|
|   1743|     3.5|       4.5|
+-------+--------+----------+
only showing top 5 rows


# Hiển thị Kết quả cuối cùng

In [13]:
# Join với movies_df để lấy tên phim
final_df = gender_stats_df.join(movies_df, "MovieID")

final_df.select(
    col("Title"),
    format_number(col("Male_Avg"), 2).alias("Male_Avg"),
    format_number(col("Female_Avg"), 2).alias("Female_Avg")
).orderBy("Title").show(200, truncate=False)

+---------------------------------------------------------+--------+----------+
|Title                                                    |Male_Avg|Female_Avg|
+---------------------------------------------------------+--------+----------+
| American Beauty (1999)                                  |3.00    |4.50      |
| Avatar (2009)                                           |5.00    |4.50      |
| Back to the Future (1985)                               |4.00    |4.00      |
| Birdman (2014)                                          |3.00    |3.00      |
| Braveheart (1995)                                       |4.50    |4.00      |
| Coco (2017)                                             |5.00    |3.50      |
| Dunkirk (2017)                                          |4.00    |3.00      |
| Fight Club (1999)                                       |5.00    |3.50      |
| Forrest Gump (1994)                                     |2.50    |5.00      |
| Gladiator (2000)                      

In [14]:
spark.stop()